In [3]:
import pandas as pd
import torch
import re
from hugchat import hugchat


In [4]:
from hugchat import hugchat

def chat_with_bot(text):
    
    chatbot = hugchat.ChatBot(cookie_path="/home/ubuntu/cookies.json")
    try:
        response = chatbot.chat(text,temperature=0.2,
                                top_k=95,
                                max_new_tokens=512,
                                )
    except:
        response = "0"
    #print(text)
    # Create a new conversation
    
    id = chatbot.new_conversation()
    chatbot.change_conversation(id)
    

    # Get conversation list 
    conversation_list = chatbot.get_conversation_list()

    return conversation_list, response


In [5]:
def read_files(desired_files_df, max_sentence_length):
    for idx in range(desired_files_df.shape[0]):
        x = desired_files_df["text"].iloc[idx]
        # print("##",x)
        sentences = re.split(r'(?<=[.!?])\s+', x)
        # print("**sentences:",sentences)
        print(len(sentences))
        # break
        # Adjust the maximum sentence length as per your requirements
        chunks = sentences
        final_chunks = []
        new_data = ""
        index = 0
        index1 = -1
        for c_index,sentence in enumerate(chunks[index:index1]):
            # print("each sentence lenght:", len(sentence.split(" ")))
            if len(sentence.split(" ")) <= max_sentence_length and len((new_data+sentence).split(" "))<= max_sentence_length:
                new_data = new_data+sentence
            else:
                # print("next setence lenght:", index, len(sentence.split(" ")))
                final_chunks.append(new_data)
                new_data = ""
                index = c_index
    return final_chunks

In [6]:
path = "/home/ubuntu/cat_poc/llms/final_data.csv"
data_frame = pd.read_csv(path)

In [7]:
data_frame

,filename,text,level
0,15031-4983-FullBook.docx,"Learner Choice, Learning Voice\n\n\nLearner Vo...",Level 1
1,15031-4984-FullBook.docx,\n\nExistentialism: A Philosophical Inquiry\n\...,Level 1
2,15031-4985-FullBook.docx,"""The editors of this volume are the top practi...",Level 1
3,15031-4986-FullBook.docx,Black Power Music!\n\nBlack Power Music!: Prot...,Level 1
4,15031-4989-FullBook.docx,Home\n\n\nHome articulates a ‘critical geograp...,Level 1
...,...,...,...
1231,15032-5774-FullBook.docx,The Japanese LGBTQ+ Community in the World\n\n...,Level 3
1232,15032-5789-FullBook.docx,The Acquisition of English Grammar and Phonolo...,Level 3
1233,15032-5829-FullBook.docx,"\n‘New concepts, new words for them, new actio...",Level 3
1234,15032-5843-FullBook.docx,Translating Rumi into the West\n\n\nFocusing o...,Level 3


In [8]:
data = data_frame[data_frame["filename"] == "15031-4983-FullBook.docx"]

In [9]:
data['text']

0    Learner Choice, Learning Voice\n\n\nLearner Vo...
Name: text, dtype: object

In [10]:
chunk_size = 400  # Adjust the chunk size as per your requirements
chunks_data = read_files(data, chunk_size)

3256


In [11]:
from langchain.text_splitter import TokenTextSplitter
text_splitter_1 = TokenTextSplitter(chunk_size=400, chunk_overlap=0)
# text_splitter_2 = TokenTextSplitter(chunk_size=150, chunk_overlap=0)
chunks = text_splitter_1.split_text(data['text'][0])


In [12]:


prompt_beginners = [
    # ''' Analyse  the following text  weather it has number_treatment give me respose in "Yes" or "No" only'''
    # ''' count no of phonetics, symbols that are there in the doccument and give me the count number '''
    '''count the number of phonetics and symbols present in the document and provide only the count just give me a number and return zero if there are none but return a number'''
    # '''Analyse  the following text  weather it has Single_or_double_quotes and give me respose in "Single" or "double" only?:''',
    # '''Analyse  the following text  weather it has Series_comma and give me respose in "Yes" or "No" only?:''',
    # '''analyse  the below text  weather it is in "American_english style" or "British english style and give me respose in "American" or "British" only?:"'''
]

Modified_chunks = []
for prompt in prompt_beginners:
    prompt_line = f'''{prompt}'''
    for chunk_i in chunks_data:
        input_text = prompt_line + chunk_i
        Modified_chunks.append(input_text)
        

In [13]:
response_text = ""
count_add = 0    
for i in range(len(Modified_chunks)):
    # print(i) 
    print(count)
    # print(Modifi
    # ed_chunks[i])
    # print(Modified_chunks.index(chunk_data))
    while True:
        conversation_list, response = chat_with_bot(Modified_chunks[i])
        # print(response)
        # numbers = sum(int(num) for num in re.findall(r'\d+', response))
        numbers = re.findall(r'\d+', response)
        count = len(numbers)
        total = sum(int(num) for num in numbers)
        print(total)
        if count > 0:
            count_add += total
            response_text += response
            break  

0
0
0
36
37
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
4
6
7
52
54
27
29
256
257
124
125
104
105
105
106
1072
1074
19
21
109
110
104
105
51
53
48
50
42
44
9
11
0
34
35
148
149
10
11
0
0
0
0
1
0
1
0
0
0
0
0
1
55
57
0
1
0
1
5
6
53
55
103
104
31
33
9
11
27
29
23
25
897
898
359
360
46
47
31
33
40
42
0


KeyboardInterrupt: 

In [ ]:
count

In [ ]:
# numbers = re.findall(r'\d+', "There are 45 phonetic characters and 1 symbol (the bullet point character) in this document")
text = "There are   phonetic characters and  symbol (the bullet point character) in this document"
# print("Count of numbers:", total)

numbers = re.findall(r'\d+', text)
count = len(numbers)
total = sum(int(num) for num in numbers) if count > 0 else 0

print("Count of numbers:", count)
print("Total sum of numbers:", total)

In [ ]:
df = pd.DataFrame(columns=['Single_or_double_quotes', 'Series_comma','American_english',"number_treatment"])

punctuational_error_count, grammatical_error_count, spelling_error_count,missing_articles_count =[],[],[],[]
for chunk_data in Modified_chunks[:]:
    conversation_list, response = chat_with_bot(chunk_data)
    print("Conversation List:", conversation_list)
    print("Response:", response)
    print()

    # Extract the numeric value using regular expressions
    # error_count = re.findall(r'\d+', response.split("\n")[0])

    if error_count:
        # Convert the extracted value to an integer
        error_count = int(error_count[0])
    else:
        # If a numeric value is not found, try to convert words to numbers
        try:
            error_count = response.split("\n")[0]
        except ValueError:
            error_count = 0
    
    print("error_count:", response)

    # Determine the column to assign the error count based on the prompt
    if "Single_or_double_quotes" in chunk_data:
        # column_name = 'punctuational error_count'
        punctuational_error_count.append(response)
    if "Series_comma" in chunk_data:
        # column_name = 'grammatical error_count'
        grammatical_error_count.append(response)
    if "American_english " in chunk_data:
        # column_name = 'spelling_error_count'
        spelling_error_count.append(response)
    if "number_treatment" in chunk_data:
        # column_name = 'missing_articles_count'
        missing_articles_count.append(error_count)
    else:
        column_name = None

    # print("column_name: ", column_name)

    # # Add the error count to the dataframe if it is not None and column_name is valid
    # if error_count is not None and column_name:
    #     if column_name in df.columns:
    #         df[column_name] = df.get(column_name, 0) + error_count
    #     else:
    #         df[column_name] = error_count

print("punc",punctuational_error_count,len(punctuational_error_count))
print("Grammer",grammatical_error_count,len(grammatical_error_count))
print("spelling",spelling_error_count,len(spelling_error_count))
# print("arti",missing_articles_count,len(missing_articles_count))
df = pd.DataFrame({'punctuational error_count':punctuational_error_count,
                   'grammatical error_count':grammatical_error_count, 
                   'spelling_error_count':spelling_error_count})
                #    'missing_articles_count':missing_articles_count})
# Print the resulting dataframe
print(df)


In [ ]:
df

In [ ]:

# Assuming your DataFrame is named 'df'
df.rename(columns={"punctuational error_count": "Single_or_double_quotes", "grammatical error_count": "Series_comma","American_english":"Style"}, inplace=True)


In [ ]:
df.head()